# Anomaly Detection in Keras

- [Source](https://github.com/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_14_03_anomaly.ipynb)
    - Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
    - For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).
    
## What Are Anomalies?
데이터 마이닝에서 이상 징후 검출(내지는 이상치 검출)은 데이터와 현저하게 다른 것으로 의심되는 항목, 사건을 식별하는 것입니다. 이상 징후는 크게 다음과 같이 분류할 수 있습니다.

Point Anomalies(포인트 이상 징후): 데이터의 단일 인스턴스가 나머지 데이터와 너무 멀리 떨어져 있는 경우입니다. 비즈니스 사례로는 "지불된 금액"을 기준으로 신용 카드 사기 감지가 존재합니다.

Contextual Anomalies(상황별 이상 징후): 이상은 상황에 따라 다릅니다. 이 유형의 Anomaly Values는 시계열 데이터에서 흔히 볼 수 있습니다. 비즈니스 활용 사례로는 연휴 기간 동안 매일 음식에 100달러를 쓰는 것은 정상이지만 그렇지 않으면 이상할 수 있다는 것입니다.

Collective Anomalies(집단 이상 징후): 일련의 데이터 인스턴스는 모두 이상 징후를 탐지하는 데 도움이 됩니다. 개별 데이터 인스턴스는 그 자체로 이상이 아닐 수 있지만 집단으로서 함께 발생하는 것을 집단 이상이라고 말합니다. 

## Import Module

In [1]:
import pandas as pd
from tensorflow.keras.utils import get_file

from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
from IPython.display import display, HTML
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

C:\Users\PARK\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\PARK\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\PARK\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\PARK\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

In [2]:
try:
    path = get_file('kddcup.data_10_percent.gz', origin='http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz')
except:
    print('Error downloading')
    raise
    
print(path) 

C:\Users\PARK\.keras\datasets\kddcup.data_10_percent.gz


## Load Data

<img src="https://hackernoon.com/hn-images/1*TYAzzTJ60x-qg5N81ElU9A.png" width="600">

### Description

네트워크 침입을 탐지하는 소프트웨어는 기업 내부자를 포함하여 무단 사용자로부터 컴퓨터 네트워크를 보호합니다. 침입 탐지기 학습 과제는 침입 또는 공격(Attack.)이라고 하는 "나쁜" 연결과 좋은 "정상적인 연결"을 구분할 수 있는 예측 모델을 구축하는 것이 목표였습니다. 

1998 DAPRA 침입 탐지 평가 프로그램은 MIT Lincoln Labs에서 총괄했습니다. 목적은 무단 네트워크 침입 연구를 조사하고 평가하는 것입니다. 군 네트워크 환경에서 시뮬레이션을 수행한 다양한 침입 유형을 포함한 데이터 셋이 이번 분석 데이터입니다.

원 자료는 7 주간의 네트워크 트패픽에서 약 4GB의 압축 바이너리 TCP(Transmission Control Protocol, 전송 제어 프로토콜) 덤프 데이터입니다. 약 5 백만 개의 레코드로 구성되어 있습니다. 또한, 2 주간의 테스트 데이터는 약 2 백만 개의 레코드로 구성되어 있습니다. 우리는 전체 데이터의 10% 샘플링 데이터를 사용하여 이번 커널을 수행할 것입니다.

각 행은 하나의 Connection으로 구성 되어있으며, Connection(연결)이란 정상적으로 정의된 시간에 시작하고 끝나는 일련의 TCP 패킷입니다. 또한, 각 연결에는 정확히 하나의 특정 공격 유형이 존재하는 Normal.(정상) 및 공격으로 라벨링 되어 있습니다.

이 데이터에서 가장 중요한 것은 훈련 데이터와 테스트 데이터의 분포 차이입니다. 테스트 데이터는 <b> 훈련 데이터와 동일한 확률 분포가 아니며, 훈련 데이터에 없는 특정 공격 유형을 포함한다는 점에 유의</b>해야합니다. 

In [3]:
df = pd.read_csv(path, header = None)

print("Read {} rows.".format(len(df)))

df.dropna(inplace = True, axis = 1)

df.columns = [
    'duration',
    'protocol_type',
    'service',
    'flag',
    'src_bytes',
    'dst_bytes',
    'land',
    'wrong_fragment',
    'urgent',
    'hot',
    'num_failed_logins',
    'logged_in',
    'num_compromised',
    'root_shell',
    'su_attempted',
    'num_root',
    'num_file_creations',
    'num_shells',
    'num_access_files',
    'num_outbound_cmds',
    'is_host_login',
    'is_guest_login',
    'count',
    'srv_count',
    'serror_rate',
    'srv_serror_rate',
    'rerror_rate',
    'srv_rerror_rate',
    'same_srv_rate',
    'diff_srv_rate',
    'srv_diff_host_rate',
    'dst_host_count',
    'dst_host_srv_count',
    'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate',
    'dst_host_serror_rate',
    'dst_host_srv_serror_rate',
    'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate',
    'outcome'
]

df.head()

Read 494021 rows.


,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,outcome
0,0,tcp,http,SF,181,5450,0,0,0,0,...,9,1.0,0.0,0.11,0.0,0.0,0.0,0.0,0.0,normal.
1,0,tcp,http,SF,239,486,0,0,0,0,...,19,1.0,0.0,0.05,0.0,0.0,0.0,0.0,0.0,normal.
2,0,tcp,http,SF,235,1337,0,0,0,0,...,29,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
3,0,tcp,http,SF,219,1337,0,0,0,0,...,39,1.0,0.0,0.03,0.0,0.0,0.0,0.0,0.0,normal.
4,0,tcp,http,SF,217,2032,0,0,0,0,...,49,1.0,0.0,0.02,0.0,0.0,0.0,0.0,0.0,normal.


## Define Function

In [9]:
def resumetable(df):
    print(f"Dataset Shape: {df.shape}")
    summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = df.isnull().sum().values    
    summary['Uniques'] = df.nunique().values

    return summary

## Exploratory Data Analysis

### Check Missing Value & Unique Value

In [10]:
resumetable(df)

Dataset Shape: (494021, 42)


,Name,dtypes,Missing,Uniques
0,duration,int64,0,2495
1,protocol_type,object,0,3
2,service,object,0,66
3,flag,object,0,11
4,src_bytes,int64,0,3300
5,dst_bytes,int64,0,10725
6,land,int64,0,2
7,wrong_fragment,int64,0,3
8,urgent,int64,0,4
9,hot,int64,0,22


데이터는 494,021개의 관측값과 42개의 컬럼으로 구성되어 있다는 것을 확인할 수 있습니다. 또한, 결측값은 존재하지 않고 고유값들은 관측값 수에 비해 그 수가 적다는 것을 알 수 있습니다.

42개의 피쳐에 대한 설명은 다음과 같습니다. 먼저 연속형 변수에 대한 설명입니다.

- duration: 연결 시간(초)
- src_bytes: 소스IP에서 대상IP까지 전송된 데이터 바이트 수
- dst_bytes: 대상IP에서 소스IP까지 전송된 데이터 바이트 수
- wrong_fragment: "wrong" 조각의 수 (?)
- urgent: 긴급 패킷 수
- hot: "hot" 표시기 수 (?)
- num_failed_logins: 실패한 로그인 시도 횟수
- num_compromised: 손상된 조건의 수(?)
- num_root: Root 접근 횟수
- num_file_creations: 파일 생성 작업 횟수
- num_shells: Shell 프롬프트 수
- num_access_files: 접근 제어 파일의 작동 횟수
- num_outbound_cmds: FTP 세션의 아웃바운드 수
- count: 지난 2초 동안 현재 연결과 동일한 호스트에 대한 연결 수

Note: 다음 피쳐는 동일한 호스트 연결에 관련된 피쳐입니다.

- serror_rate: 'SYN' 에러가 발생한 연결 비율
- rerror_rate: 'REJ' 에러가 발생한 연결 비율
- same_srv_rate: 동일한 서비스에 대한 연결 비율
- diff_srv_rate: 다른 서비스에 대한 연결 비율

Note: 다음 피쳐는 동일한 서비스 연결에 관련된 피쳐입니다.

- srv_count: 지난 2초 동안 현재 연결과 동일한 서비스에 대한 연결 수
- srv_serror: 'SYN' 에러가 발생한 연결 비율
- srv_rerror_rate: 'REJ' 에러가 발생한 연결 비율
- srv_diff_host_rate: 다른 호스트에 대한 연결 비율

이제, 이산형 변수에 대한 설명입니다.

- protocol_type: 프로토콜 유형
- service: 대상IP의 네트워크 서비스
- flag: 연결의 정상 또는 오류 상태
- land: 연결이 동일한 호스트 및 포트에서 연결되는 경우. 1 = Y, 0 = N
- logged_in: 로그인 성공 여부 1 = Y, 0 = N
- root_shell: "Root Shell"이 확보 여부 1 = Y, 0 = N
- su_attempted: "su root" 명령 시도 여부 (sudo 등) 1 = Y, 0 = N
- is_hot_login: 로그인이 'hot'에 속하는 지에 대한 여부 1 = Y, 0 = N
- is_guest_login: guest 로그인 여부 1 = Y, 0 = N

### Summary Statistics

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
duration,494021.0,47.979302,707.746472,0.0,0.00,0.0,0.00,58329.0
src_bytes,494021.0,3025.610296,988218.101045,0.0,45.00,520.0,1032.00,693375640.0
dst_bytes,494021.0,868.532425,33040.001252,0.0,0.00,0.0,0.00,5155468.0
land,494021.0,0.000045,0.006673,0.0,0.00,0.0,0.00,1.0
wrong_fragment,494021.0,0.006433,0.134805,0.0,0.00,0.0,0.00,3.0
urgent,494021.0,0.000014,0.005510,0.0,0.00,0.0,0.00,3.0
hot,494021.0,0.034519,0.782103,0.0,0.00,0.0,0.00,30.0
num_failed_logins,494021.0,0.000152,0.015520,0.0,0.00,0.0,0.00,5.0
logged_in,494021.0,0.148247,0.355345,0.0,0.00,0.0,0.00,1.0
num_compromised,494021.0,0.010212,1.798326,0.0,0.00,0.0,0.00,884.0


### Label
데이터의 라벨인 outcome의 라벨은 총 24개 가지의 공격 유형이 포함되어 있습니다. 

공격은 크게 4 가지 범주로 분류됩니다.
- DOS: 서비스 거부
- R2L: 원격 컴퓨터에서 무단 액세스
- U2R: 로컬 관리자 계정 권한에 대한 무단 액세스
- probing: 감시 및 기타 프로빙 (예: port scan)

DOS 범주에 들어가는 공격 유형은 다음과 같습니다. 
- back
- land
- neptune
- smurf

R2L 범주에 들어가는 공격 유형은 다음과 같습니다.
- ftp_write
- guess_passwd
- imap
- multihop
- phf
- spy
- warezclinet
- warezmaster



In [14]:
df['outcome'].unique()

array(['normal.', 'buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.'], dtype=object)

# TO-DO

In [ ]:
df_columns = df.columns.tolist()#.tolist().delete('protocol_type','service','flag')
categorical_columns_index = [1,2,3,6,11,20,21,41]

numerical_columns = np.delete(df_columns, categorical_columns_index)
categorical_columns = ['protocol_type', 'service', 'flag', 'land',
                       'is_host_login', 'is_guest_login']

In [ ]:
std = StandardScaler()

for col in numerical_columns:
    df[col] = std.fit_transform(df[[col]])

def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    df.drop(name, axis = 1, inplace = True)

In [ ]:
encode_text_dummy(df, 'protocol_type')
encode_text_dummy(df, 'service')
encode_text_dummy(df, 'flag')
encode_text_dummy(df, 'logged_in')
encode_text_dummy(df, 'is_host_login')
encode_text_dummy(df, 'is_guest_login')

df.dropna(inplace = True, axis =1)
df.head()

In [ ]:
normal_mask = df['outcome'] == 'normal.'
attack_mask = df['outcome'] != 'normal.'

df.drop(columns = 'outcome', inplace = True)

df_normal = df[normal_mask]
df_attack = df[attack_mask]

print("Normal Count: ", len(df_normal))
print("Attack Count:", len(df_attack))

In [ ]:
x_normal = df_normal.values
x_attack = df_attack.values

x_normal_train, x_normal_test = train_test_split(x_normal, test_size = 0.25, random_state = 42)

print("Normal Train Count:", len(x_normal_train))
print("Normal Test Count:", len(x_normal_test))

In [ ]:
model = Sequential()
model.add(Dense(25, input_dim = x_normal.shape[1], activation = 'relu'))
model.add(Dense(3, activation = 'relu'))
model.add(Dense(25, activation = 'relu'))
model.add(Dense(x_normal.shape[1]))

model.compile(loss = 'mean_squared_error', optimizer = 'adam')
model.summary()

In [ ]:
model.fit(x_normal_train, x_normal_train, batch_size = 50,
          validation_split = 0.25,
          verbose = 1, epochs = 30,
         callbacks = [EarlyStopping(monitor = 'val_loss', patience = 3)])

In [ ]:
pred = model.predict(x_normal_test)
score1 = np.sqrt(metrics.mean_squared_error(pred, x_normal_test))
pred = model.predict(x_normal)
score2 = np.sqrt(metrics.mean_squared_error(pred, x_normal))
pred = model.predict(x_attack)
score3 = np.sqrt(metrics.mean_squared_error(pred, x_attack))

print(f"Insample Normal Score (RMSE): {score1}".format(score1))
print(f"Out of Sample Normal Score (RMSE): {score2}")
print(f"Attack Underway Score (RMSE): {score3}")